# KNN Regression

KNN in our problem might not be the best choice - variables highly skewed! But we will do our best!

We will apply following approach in case of KNN:
 * we will perform feature engineering - standardization
 * then we will find "good enough" parameters (in CV) to proceed feature selection procedure and after than we will have groups of feature candidates
 * then we will tune hyperparameters for each group of variables (in CV) - we will obtain couple of models
 * then we will compare all models based on so called "proper CV" and we will fit and picke the winner!

We are aware of potential data leakage in case of usage KFold CV without time-series problem handling (in point 2 and 3). 

*To be honest in this problem it is not a big deal - based on our experience and we treat it like a feature!*

During the last step of our procedure we will verify previous analysis based on "proper CV" which handle time-series properties!!! So during hyperparameters tuning we will use different CV (we fight against data leakage).

### Dependencies loading

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.inspection import permutation_importance
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from ReliefF import ReliefF
import pickle
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

### Data loading

In [2]:
df = pd.read_csv("../data/train_fe.csv", index_col=0)

In [3]:
fr = pd.read_excel("../data/feature_ranking.xlsx", index_col=0)

### Feature engineering for KNN model

We have to standardize our variables. We will use range standardization (Min Max Scaler) because we have got dummies! We gave every variable a chance to have the same impact on the model.

In [4]:
print(df.columns.tolist())

['Ticker', 'Nazwa2', 'rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'ta_log', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_c

In [5]:
columns = ['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'ta_log', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_cat_(8108.5, inf]', 'str_cat_(0.0875, 0.192]', 'str_cat_(0.192, 0.28]', 'str_cat_(0.28, inf]', 'xrd_exists', 'ni_profit', 'ni_profit_20000', 'ppent_sqrt', 'intant_sqrt', 'dlc_cat_(42.262, 176.129]', 'dlc_cat_(176.129, 200.9]', 'dlc_cat_(200.9, inf]', 'dltt_cat_(39.38, 327.85]', 'dltt_cat_(327.85, 876.617]', 'dltt_cat_(876.617, inf]', 'capex_cat_(7.447, 79.55]', 'capex_cat_(79.55, 5451.0]', 'capex_cat_(5451.0, inf]', 'revenue_cat_(0.174, 1248.817]', 'revenue_cat_(1248.817, 4233.587]', 'revenue_cat_(4233.587, inf]', 'cce_cat_(5.619, 63.321]', 'cce_cat_(63.321, inf]', 'adv_cat_(0.3, 874.5]', 'adv_cat_(874.5, inf]', 'diff_positive', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'adv_expenditure_positive', 'diff_dta', 'cfc_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']

In [6]:
standardization = list()
not_standardization = list()
for i in columns:
    if df[i].nunique() > 2:
        standardization.append(i)
    else:
        not_standardization.append(i)

In [7]:
print(standardization)

['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'capex2_scaled', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'ta_log', 'ppent_sqrt', 'intant_sqrt', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']


In [8]:
print(not_standardization)

['cfc', 'dta', 'y_e_p_polity', 'y_BR_Democracy', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_cat_(8108.5, inf]', 'str_cat_(0.0875, 0.192]', 'str_cat_(0.192, 0.28]', 'str_cat_(0.28, inf]', 'xrd_exists', 'ni_profit', 'ni_profit_20000', 'dlc_cat_(42.262, 176.129]', 'dlc_cat_(176.129, 200.9]', 'dlc_cat_(200.9, inf]', 'dltt_cat_(39.38, 327.85]', 'dltt_cat_(327.85, 876.617]', 'dltt_cat_(876.617, inf]', 'capex_cat_(7.447, 79.55]', 'capex_cat_(79.55, 5451.0]', '

In [9]:
standardization.remove("etr")

In [10]:
standardization.append("y_e_p_polity")

In [11]:
print(standardization)

['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'capex2_scaled', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'ta_log', 'ppent_sqrt', 'intant_sqrt', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past', 'y_e_p_polity']


In [12]:
scaler = MinMaxScaler()
scaler.fit(df[standardization])
df[standardization] = scaler.transform(df[standardization])

In [13]:
pickle.dump(scaler, open("final_models/minmaxscaler.sav", 'wb'))

In [14]:
# df[standardization]= scaler.fit_transform(df[standardization])

Double check. Everything seems to be good :-) 

In [15]:
df[columns].describe().T["min"].unique()

array([0., 1.])

In [16]:
df[columns].describe().T["max"].unique()

array([1., 1., 1.])

### Searching for "good enough" model to feature selection

Let's use top 10 variables proposed by Mutual Information!

In [17]:
var = fr.mi_score.sort_values(ascending=False).index.tolist()[0:10]

In [18]:
print(var)

['etr_y_past', 'etr_y_ma', 'txt', 'diff', 'ni', 'pi', 'intant', 'intant_sqrt', 'ta', 'dlc']


In [19]:
df.shape[0]**(0.5)

63.190189111918315

In [20]:
param = {"n_neighbors":[5,7,10,12,15,25,40,50,100], "weights": ["uniform", "distance"], "metric":["minkowski", "manhattan", "chebyshev"], "p":[1,2]}

In [21]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [22]:
model = KNeighborsRegressor()
grid_CV = GridSearchCV(model, param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1)
grid_CV.fit(df.loc[:,var].values, df.loc[:,"etr"].values.ravel())

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'metric': ['minkowski', 'manhattan', 'chebyshev'],
                         'n_neighbors': [5, 7, 10, 12, 15, 25, 40, 50, 100],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=make_scorer(mean_squared_error, greater_is_better=False),
             verbose=0)

In [23]:
grid_CV.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=1,
                    weights='uniform')

In [24]:
grid_CV.cv_results_

{'mean_fit_time': array([0.01259265, 0.01259317, 0.01199236, 0.01159225, 0.01219239,
        0.01279182, 0.01259208, 0.01199365, 0.01219258, 0.01259203,
        0.01359119, 0.01399083, 0.01559086, 0.01239247, 0.01319137,
        0.0119925 , 0.01219277, 0.01259198, 0.01179218, 0.01179233,
        0.01179194, 0.01219091, 0.01379128, 0.01159315, 0.01239195,
        0.012991  , 0.01199193, 0.01199217, 0.01239195, 0.01499119,
        0.01539078, 0.01419106, 0.01399083, 0.01439109, 0.01459088,
        0.01459146, 0.01499114, 0.01519046, 0.01419172, 0.01499085,
        0.01439109, 0.01439157, 0.01499071, 0.01479096, 0.01599026,
        0.01419129, 0.01399126, 0.01479096, 0.01439085, 0.01678934,
        0.01419158, 0.01459141, 0.01519046, 0.01459117, 0.01578951,
        0.01439066, 0.01439133, 0.01439104, 0.01898847, 0.01618967,
        0.01459045, 0.01479063, 0.0143908 , 0.01439056, 0.01419129,
        0.01519017, 0.01459022, 0.01439123, 0.01559043, 0.01419053,
        0.01459041, 0.01419153,

Right now (temporary) we will this hyperparameters as the best one:

In [25]:
grid_CV.best_estimator_

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=1,
                    weights='uniform')

### Feature selection for KNN

We would like to base our feature selection on:
 * feature ranking
 * forward elimination
 * relief - special feature selection model for KNN found during Michał Woźniak & Michał Dyczko undergrad. research

#### Feature ranking

In [26]:
fr.sort_values("mi_score", ascending=False, inplace=True)

In [27]:
fr.head()

,mi_score,sign_fscore,sign_fscore_0_1,corr,EN_coef,boruta_rank
etr_y_past,1.007860,1.304040e-84,1,0.520405,NaN,1
etr_y_ma,0.825573,2.473770e-125,1,0.526827,NaN,1
txt,0.633965,5.246456e-13,1,0.368732,1.466269e-05,1
diff,0.628929,2.257712e-02,1,-0.291716,NaN,1
ni,0.616573,1.747230e-09,1,0.263458,-3.442000e-07,2


In [28]:
br_features = fr[fr.boruta_rank.isin([1,2,3])].index.tolist()

In [29]:
mi_features = fr.iloc[0:20].index.tolist()

In [30]:
mi_features_25 = fr.iloc[0:25].index.tolist()

In [31]:
mi_features_35 = fr.iloc[0:35].index.tolist()

In [32]:
mi_features_50 = fr.iloc[0:50].index.tolist()

In [33]:
fr["corr_abs"] = np.abs(fr["corr"])
fr.sort_values("corr_abs", ascending=False, inplace=True)
corr_features = fr.iloc[0:20].index.tolist()

We will use our intuition and create two additional benchmark sets of variables:

In [34]:
benchmark = ['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']

In [35]:
benchmark2 = ['ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 
              'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']

#### Forward elimination

In [36]:
forward_elimination = ['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'ta_log', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_cat_(8108.5, inf]', 'str_cat_(0.0875, 0.192]', 'str_cat_(0.192, 0.28]', 'str_cat_(0.28, inf]', 'xrd_exists', 'ni_profit', 'ni_profit_20000', 'ppent_sqrt', 'intant_sqrt', 'dlc_cat_(42.262, 176.129]', 'dlc_cat_(176.129, 200.9]', 'dlc_cat_(200.9, inf]', 'dltt_cat_(39.38, 327.85]', 'dltt_cat_(327.85, 876.617]', 'dltt_cat_(876.617, inf]', 'capex_cat_(7.447, 79.55]', 'capex_cat_(79.55, 5451.0]', 'capex_cat_(5451.0, inf]', 'revenue_cat_(0.174, 1248.817]', 'revenue_cat_(1248.817, 4233.587]', 'revenue_cat_(4233.587, inf]', 'cce_cat_(5.619, 63.321]', 'cce_cat_(63.321, inf]', 'adv_cat_(0.3, 874.5]', 'adv_cat_(874.5, inf]', 'diff_positive', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'adv_expenditure_positive', 'diff_dta', 'cfc_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']
forward_elimination.remove("ta_log")
forward_elimination.remove("ppent_sqrt")
forward_elimination.remove("intant_sqrt")
forward_elimination.remove("roa")
forward_elimination.remove("lev")
forward_elimination.remove("intan")
forward_elimination.remove("rd_sqrt")
forward_elimination.remove("ppe")
forward_elimination.remove("cash_holdings_sqrt")

In [37]:
candidates_withoud_discr = [i for i in forward_elimination if "]" not in i]

In [38]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=1,
                    weights='uniform')

In [39]:
sf = SFS(model, 
           k_features=(5,15), 
           forward=True, 
           floating=False, 
           verbose=0,
           scoring=mse,
           cv=5)

sffit = sf.fit(df.loc[:,candidates_withoud_discr].values, df.loc[:,"etr"].values.ravel())

sf_features = df.loc[:,candidates_withoud_discr].columns[list(sffit.k_feature_idx_)]

sf_features

Index(['rok', 'str', 'xrd', 'dlc', 'dltt', 'diff', 'y_BR_Democracy',
       'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'sale_ma'],
      dtype='object')

In [56]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=1,
                    weights='uniform')

In [57]:
sf = SFS(model, 
           k_features=(5,15), 
           forward=True, 
           floating=False, 
           verbose=0,
           scoring=mse,
           cv=5, n_jobs=-1)

sffit = sf.fit(df.loc[:,forward_elimination].values, df.loc[:,"etr"].values.ravel())

sf_features2 = df.loc[:,forward_elimination].columns[list(sffit.k_feature_idx_)]

sf_features2

Index(['y_e_p_polity', 'sektor_energy', 'sektor_health care',
       'sektor_real estate', 'sektor_technology', 'gielda_4',
       'pi_cat_(-8975.0, -1.523]', 'str_cat_(0.192, 0.28]',
       'dltt_cat_(327.85, 876.617]', 'dltt_cat_(876.617, inf]',
       'adv_cat_(874.5, inf]'],
      dtype='object')

In [58]:
sf_one_more = ['rok', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta']

In [59]:
sf = SFS(model, 
           k_features=(5,10), 
           forward=True, 
           floating=False, 
           verbose=0,
           scoring=mse,
           cv=5, n_jobs=-1)

sffit = sf.fit(df.loc[:,sf_one_more].values, df.loc[:,"etr"].values.ravel())

sf_features3 = df.loc[:,sf_one_more].columns[list(sffit.k_feature_idx_)]

sf_features3

Index(['diff', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2'], dtype='object')

#### Relief

In [60]:
relief = ['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'rd', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_cat_(8108.5, inf]', 'str_cat_(0.0875, 0.192]', 'str_cat_(0.192, 0.28]', 'str_cat_(0.28, inf]', 'xrd_exists', 'ni_profit', 'ni_profit_20000', 'dlc_cat_(42.262, 176.129]', 'dlc_cat_(176.129, 200.9]', 'dlc_cat_(200.9, inf]', 'dltt_cat_(39.38, 327.85]', 'dltt_cat_(327.85, 876.617]', 'dltt_cat_(876.617, inf]', 'capex_cat_(7.447, 79.55]', 'capex_cat_(79.55, 5451.0]', 'capex_cat_(5451.0, inf]', 'revenue_cat_(0.174, 1248.817]', 'revenue_cat_(1248.817, 4233.587]', 'revenue_cat_(4233.587, inf]', 'cce_cat_(5.619, 63.321]', 'cce_cat_(63.321, inf]', 'adv_cat_(0.3, 874.5]', 'adv_cat_(874.5, inf]', 'diff_positive', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'ppe_clip', 'adv_expenditure_positive', 'diff_dta', 'cfc_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']

In [61]:
fs = ReliefF(n_neighbors=30, n_features_to_keep=5)

In [62]:
fs.fit(df.loc[:,relief].values, df.loc[:,"etr"].values.ravel())

In [63]:
relief1 = df.loc[:,relief].iloc[:,fs.top_features[0:10]].columns.tolist()

In [64]:
relief2 = df.loc[:,relief].iloc[:,fs.top_features[0:15]].columns.tolist()

In [65]:
relief3 = df.loc[:,relief].iloc[:,fs.top_features[0:20]].columns.tolist()

### Hyperparametes Tunning for each group of variables

In [66]:
param = {"n_neighbors":[5,7,10,12,15,25,40,50], "weights": ["uniform", "distance"], "metric":["minkowski", "manhattan", "chebyshev"], "p":[1,2]}
mse = make_scorer(mean_squared_error, greater_is_better=True)

In [67]:
def cv_proc(var):
    model = KNeighborsRegressor()
    grid_CV = GridSearchCV(model, param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1)
    grid_CV.fit(df.loc[:,var].values, df.loc[:,"etr"].values.ravel())
    print(grid_CV.best_estimator_)
    print(grid_CV.best_score_)

In [68]:
cv_proc(benchmark)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
0.025688847086774234


In [69]:
cv_proc(benchmark2)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
0.023681603682248326


In [70]:
cv_proc(mi_features_25)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.023687438936828415


In [71]:
cv_proc(mi_features_35)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.024132263841463268


In [72]:
cv_proc(mi_features_50)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.02408388165995253


In [73]:
cv_proc(br_features)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='distance')
0.021729978128990298


In [74]:
cv_proc(mi_features)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.023515242037715163


In [75]:
cv_proc(corr_features)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.023940236434834748


In [76]:
cv_proc(sf_features)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.022174924760890294


In [77]:
cv_proc(sf_features2)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
0.03854412216385179


In [78]:
cv_proc(sf_features3)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='distance')
0.030736984076166145


In [79]:
cv_proc(relief1)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.02808229700930199


In [80]:
cv_proc(relief2)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.02782631490123545


In [81]:
cv_proc(relief3)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
0.028517759404415605


### Final models comparison - winner obtaining

We would like to fight against data leakage in our CV, so we will treat it like a panel problem with a rolling window. We now based on our experience that this kind of approach is crucial to fight against overfitting.

Sliding window:
 * T: 2005 - 2008; V: 2009
 * T: 2005 - 2009; V: 2010
 * T: 2005 - 2010; V: 2011
 * ...

In [82]:
df = df.sort_values(by="rok").reset_index(drop=True)

In [83]:
def proper_CV(x, y, model, display_res = False):
    train_score = list()
    valid_score = list()
    train_indexes = [0, 1452]
    valid_indexes = [1452, 1815]
    for i in range(0,6):
        train_x =  x[x.index.isin(range(train_indexes[0],train_indexes[1]))]
        train_y =  y[y.index.isin(range(train_indexes[0],train_indexes[1]))]
        valid_x =  x[x.index.isin(range(valid_indexes[0],valid_indexes[1]))]
        valid_y =  y[y.index.isin(range(valid_indexes[0],valid_indexes[1]))]

        model.fit(train_x.values, train_y.values.ravel())
        
        pred_y_train = model.predict(train_x.values)
        rmse = np.sqrt(mean_squared_error(train_y, pred_y_train))
        train_score.append(rmse)

        pred_y_val = model.predict(valid_x.values)
        rmse = np.sqrt(mean_squared_error(valid_y, pred_y_val))
        valid_score.append(rmse)
        
        train_indexes = [0, valid_indexes[1]]
        valid_indexes = [train_indexes[1], valid_indexes[1]+363]
    
    if display_res == True:
        view = pd.DataFrame([train_score, valid_score]).T.rename(columns = {0:"cv_train", 1:"cv_val"})
        display(view)
        return train_score, valid_score, view
    else:
        return train_score, valid_score

In [101]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
var = benchmark
cv_output0 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.125956,0.151071
1,0.123942,0.146469
2,0.124273,0.157191
3,0.125281,0.148774
4,0.124349,0.135068
5,0.122619,0.148496


In [102]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')

var = benchmark2
cv_output1 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.123381,0.149848
1,0.121944,0.145164
2,0.121195,0.157434
3,0.122649,0.151020
4,0.122394,0.142104
5,0.122157,0.147819


In [103]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')
var = br_features
cv_output2 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.117684,0.153403
1,0.117678,0.136081
2,0.115893,0.165368
3,0.119046,0.151590
4,0.119095,0.135799
5,0.117716,0.146451


In [104]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = mi_features
cv_output3 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.024271,0.162502
1,0.025683,0.143987
2,0.028871,0.169731
3,0.028130,0.156194
4,0.026329,0.130178
5,0.025827,0.141895


In [105]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
var = corr_features
cv_output4 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.123424,0.156191
1,0.125023,0.138310
2,0.122805,0.157209
3,0.123024,0.153019
4,0.122702,0.136384
5,0.122398,0.145986


In [106]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
var = sf_features
cv_output5 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.117680,0.157800
1,0.116522,0.146221
2,0.116989,0.159230
3,0.119275,0.150756
4,0.119939,0.134568
5,0.118065,0.145907


In [107]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = sf_features2
cv_output6 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.154031,0.154844
1,0.157663,0.152656
2,0.156967,0.165148
3,0.160991,0.162095
4,0.160958,0.155972
5,0.160401,0.162150


In [108]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = sf_features3
cv_output7 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.156305
1,0.030226,0.171961
2,0.034041,0.169983
3,0.032214,0.171439
4,0.030203,0.160629
5,0.029170,0.155571


In [109]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = relief1
cv_output8 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.149546
1,0.030226,0.155123
2,0.033734,0.162585
3,0.032146,0.160728
4,0.030527,0.142725
5,0.029191,0.157520


In [110]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = relief2
cv_output9 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.158226
1,0.030226,0.149294
2,0.033435,0.159193
3,0.032709,0.156948
4,0.030148,0.143187
5,0.029186,0.148951


In [111]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = relief3 
cv_output10 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.161661
1,0.030226,0.161281
2,0.033404,0.159285
3,0.032693,0.163421
4,0.030143,0.145755
5,0.029224,0.153538


In [112]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='chebyshev',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = mi_features_25 
cv_output11 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.024263,0.162663
1,0.025652,0.141275
2,0.028848,0.164978
3,0.027889,0.154550
4,0.026176,0.130554
5,0.025451,0.141324


In [113]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='distance')
var = mi_features_35 
cv_output12 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.023087,0.152087
1,0.024506,0.142924
2,0.027961,0.156399
3,0.027103,0.149464
4,0.025429,0.133746
5,0.024774,0.137505


In [114]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=1,
                    weights='uniform')
var = mi_features_50 
cv_output13 = proper_CV(df.loc[:,var], df.loc[:,"etr"], model, display_res=True)

,cv_train,cv_val
0,0.120017,0.143887
1,0.120044,0.142002
2,0.119194,0.156382
3,0.120959,0.150758
4,0.120496,0.135599
5,0.119601,0.139886


In [115]:
pd.DataFrame([cv_output0[2].mean().tolist(),
cv_output1[2].mean().tolist(), 
cv_output2[2].mean().tolist(),
cv_output3[2].mean().tolist(),
cv_output4[2].mean().tolist(),
cv_output5[2].mean().tolist(),
cv_output6[2].mean().tolist(),
cv_output7[2].mean().tolist(),
cv_output8[2].mean().tolist(),
cv_output9[2].mean().tolist(),
cv_output10[2].mean().tolist(),
cv_output11[2].mean().tolist(),
cv_output12[2].mean().tolist(),
cv_output13[2].mean().tolist(),], columns=["train_mean", "test_mean"])

,train_mean,test_mean
0,0.124403,0.147845
1,0.122287,0.148898
2,0.117852,0.148115
3,0.026519,0.150748
4,0.123229,0.147850
5,0.118078,0.149080
6,0.158502,0.158811
7,0.030671,0.164315
8,0.030666,0.154705
9,0.030646,0.152633


In [116]:
pd.DataFrame([cv_output0[2].std().tolist(),
cv_output1[2].std().tolist(), 
cv_output2[2].std().tolist(),
cv_output3[2].std().tolist(),
cv_output4[2].std().tolist(),
cv_output5[2].std().tolist(),
cv_output6[2].std().tolist(),
cv_output7[2].std().tolist(),
cv_output8[2].std().tolist(),
cv_output9[2].std().tolist(),
cv_output10[2].std().tolist(),
cv_output11[2].std().tolist(),
cv_output12[2].std().tolist(),
cv_output13[2].std().tolist(),], columns=["train_std", "test_std"])

,train_std,test_std
0,0.001149,0.007269
1,0.000730,0.005282
2,0.001173,0.011287
3,0.001697,0.014665
4,0.000943,0.009053
5,0.001316,0.009063
6,0.002789,0.004975
7,0.002128,0.007688
8,0.002010,0.007438
9,0.002035,0.006432


13th models seems to be the best one! We see that our intuition was quite good - binary variables should be removed!

In [117]:
print(mi_features_50)

['etr_y_past', 'etr_y_ma', 'txt', 'diff', 'ni', 'pi', 'intant', 'intant_sqrt', 'ta', 'dlc', 'revenue', 'roa_clip', 'roa', 'capex', 'diff_ma', 'ta_log', 'cce', 'dltt', 'intan_past', 'sale', 'intan_ma', 'intan', 'ppent', 'roa_past', 'intan_pow2', 'cash_holdings_sqrt', 'cash_holdings', 'sale_past', 'ppent_sqrt', 'ppe_clip', 'ppe_past', 'roa_ma', 'ppe', 'lev_ma', 'sale_ma', 'capex2', 'cash_holdings_ma', 'capex2_scaled', 'lev_past', 'lev', 'ppe_ma', 'lev_sqrt', 'cash_holdings_past', 'str', 'str_cat_(0.0875, 0.192]', 'WB_GDPpc', 'rr_per_country', 'WB_GDPgrowth', 'WB_Inflation', 'str_cat_(0.28, inf]']


### Fit final model and save it

In [118]:
model = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=1,
                    weights='uniform')
model.fit(df.loc[:,mi_features_50].values, df.loc[:,"etr"].values.ravel())

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=50, p=1,
                    weights='uniform')

In [119]:
filename = 'final_models/knn.sav'

In [120]:
pickle.dump(model, open(filename, 'wb'))